# Getting started with 2-variable nonsmooth Rosenbrock objective function

This notebook contains examples of how to solve optimization problem with 2-variable nonsmooth Rosenbrock objective function, which subject to simple bound constraints.

For more details, please check the documentation website https://pygranso.readthedocs.io/en/latest/

1. Import all necessary modules and add PyGRANSO src folder to system path.

In [1]:
import time
import torch
import sys
## Adding PyGRANSO directories. Should be modified by user
sys.path.append('/home/buyun/Documents/GitHub/PyGRANSO')
from pygranso import pygranso
from pygransoStruct import Options, Data, GeneralStruct

2. Spceify torch device, optimization variables, and corresponding objective and constrained function.

Note: please strictly follow the format of evalObjFunction and combinedFunction, which will be used in the PyGRANSO main algortihm. *X_struct* and *data_in* are always required.

In [2]:
device = torch.device('cpu')
# variables and corresponding dimensions.
var_in = {"x1": [1,1], "x2": [1,1]}


def evalObjFunction(X_struct,data_in = None):
    x1 = X_struct.x1
    x2 = X_struct.x2
    
    # enable autodifferentiation
    x1.requires_grad_(True)
    x2.requires_grad_(True)
    # objective function
    f = (8 * abs(x1**2 - x2) + (1 - x1)**2)
    return f

def combinedFunction(X_struct,data_in = None):
    x1 = X_struct.x1
    x2 = X_struct.x2
    # enable autodifferentiation
    x1.requires_grad_(True)
    x2.requires_grad_(True)
    
    # objective function
    f = (8 * abs(x1**2 - x2) + (1 - x1)**2)

    # inequality constraint, matrix form
    ci = GeneralStruct()
    ci.c1 = (2**0.5)*x1-1  
    ci.c2 = 2*x2-1 

    # equality constraint 
    ce = None

    return [f,ci,ce]

obj_eval_fn = lambda X_struct,data_in = None : evalObjFunction(X_struct,data_in = None)
comb_fn = lambda X_struct,data_in = None : combinedFunction(X_struct,data_in = None)

3. Specify user-defined options for PyGRANSO algorithm

In [3]:
opts = Options()
opts.QPsolver = 'osqp'
opts.x0 = torch.ones((2,1), device=device, dtype=torch.double)

4. Run main algorithm

In [4]:
start = time.time()
soln = pygranso(combinedFunction = comb_fn, objEvalFunction = obj_eval_fn,var_dim_map = var_in, torch_device = device, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))
print(soln.final.x)

/home/buyun/anaconda3/envs/cuosqp_pygranso/lib/python3.9/site-packages/torch/autograd/__init__.py:147: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/cuda/CUDAFunctions.cpp:115.)
  Variable._execution_engine.run_backward(
/home/buyun/anaconda3/envs/cuosqp_pygranso/lib/python3.9/site-packages/osqp/utils.py:122: UserWarning: Converting sparse A to a CSC (compressed sparse column) matrix. (It may take a while...)
  warn("Converting sparse A to a CSC " +




╔═════ QP SOLVER NOTICE ══════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible   ║
║  interface, as defined by osqp and Gurobi...                                        ║
╚═════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
Problem specifications:                                                                                          ║ 
 # of variables                     :   2                                                                        ║ 
 # of inequality constraints        :   2                                                                        ║ 
 # of equality constraints          :   0                                                                        ║ 
═════╦═══════════════════════════╦════════════════╦════════════════